# Timbre Painting in DDSP

## Basic Upsampler

In [ ]:
%load_ext autoreload
%autoreload 2

# Extract some f0
from ddsp.synths import BasicUpsampler
import numpy as np
from matplotlib import pyplot as plt
from ddsp.colab.jupyter_utils import show_audio

sample_rate = 16000
n_samples = 4*sample_rate
synth = BasicUpsampler(n_samples)

f0_hz = np.linspace(400, 800, 100).reshape([1,-1,1])
amplitudes = np.abs(np.sin(np.linspace(0, 2*np.pi, 100))).reshape([1,-1,1])

wav = synth.get_signal(amplitudes, f0_hz)
show_audio(wav, focus_points=[0.45, 0.8], focus_windows=[2000, 2000])

## Basic Upsampler + ParallelWaveGANUpsampler

In [ ]:
from ddsp.training.decoders import TimbrePaintingDecoder

decoder = TimbrePaintingDecoder(name='tpd', input_keys=('amplitudes', 'f0_hz'))
batch = {
    'f0_hz': f0_hz,
    'amplitudes': amplitudes
}
controls = decoder(batch)


In [ ]:
wav = controls['audio_tensor'].numpy().squeeze()
show_audio(wav, focus_points=[0.05, 0.95], focus_windows=[2000, 2000])

## Discriminator

In [ ]:
from ddsp.training import discriminator

critic = discriminator.ParallelWaveGANDiscriminator(input_keys=['audio_tensor', 'f0_hz', 'amplitudes'])
critic_score = critic(controls)

critic_score

## Gan Autoencoder

In [ ]:
import ddsp
from ddsp.training import models, preprocessing, decoders, discriminator
from ddsp import synths



dag = [(synths.TensorToAudio(), ['audio_tensor'])]

ae = models.Autoencoder(
    preprocessor=None,
    encoder=None,
    decoder=decoders.TimbrePaintingDecoder(name='tpd', input_keys=('amplitudes', 'f0_hz')),
    processor_group=ddsp.processors.ProcessorGroup(dag=dag, name='processor_group'),
    discriminator=discriminator.ParallelWaveGANDiscriminator(input_keys=['discriminator_audio', 'f0_hz', 'amplitudes']),
    losses=[]
)

batch = {
    'f0_hz': f0_hz,
    'amplitudes': amplitudes,
    'audio': np.random.normal(0,1,size=n_samples)
}
outputs = ae(batch)

outputs.keys()

# Training